## Problem 2

This problem uses the OJ data set in the ISLR package.

**Description** 

The data contains 1070 purchases where the customer either purchased Citrus Hill or Minute Maid Orange Juice. A number of characteristics of the customer and product are recorded.

In [1]:
# Attaching the dataset and then exploring the data
library(ISLR)
attach(OJ)
names(OJ)

[1] "Purchase"       "WeekofPurchase" "StoreID"        "PriceCH"       
 [5] "PriceMM"        "DiscCH"         "DiscMM"         "SpecialCH"     
 [9] "SpecialMM"      "LoyalCH"        "SalePriceMM"    "SalePriceCH"   
[13] "PriceDiff"      "Store7"         "PctDiscMM"      "PctDiscCH"     
[17] "ListPriceDiff"  "STORE"

In [2]:
head(OJ)

Purchase,WeekofPurchase,StoreID,PriceCH,PriceMM,DiscCH,DiscMM,SpecialCH,SpecialMM,LoyalCH,SalePriceMM,SalePriceCH,PriceDiff,Store7,PctDiscMM,PctDiscCH,ListPriceDiff,STORE
CH,237,1,1.75,1.99,0.00,0.0,0,0,0.500000,1.99,1.75,0.24,No,0.000000,0.000000,0.24,1
CH,239,1,1.75,1.99,0.00,0.3,0,1,0.600000,1.69,1.75,-0.06,No,0.150754,0.000000,0.24,1
CH,245,1,1.86,2.09,0.17,0.0,0,0,0.680000,2.09,1.69,0.40,No,0.000000,0.091398,0.23,1
MM,227,1,1.69,1.69,0.00,0.0,0,0,0.400000,1.69,1.69,0.00,No,0.000000,0.000000,0.00,1
CH,228,7,1.69,1.69,0.00,0.0,0,0,0.956535,1.69,1.69,0.00,Yes,0.000000,0.000000,0.00,0
CH,230,7,1.69,1.99,0.00,0.0,0,1,0.965228,1.99,1.69,0.30,Yes,0.000000,0.000000,0.30,0


In [3]:
summary(OJ)

 Purchase WeekofPurchase     StoreID        PriceCH         PriceMM     
 CH:653   Min.   :227.0   Min.   :1.00   Min.   :1.690   Min.   :1.690  
 MM:417   1st Qu.:240.0   1st Qu.:2.00   1st Qu.:1.790   1st Qu.:1.990  
          Median :257.0   Median :3.00   Median :1.860   Median :2.090  
          Mean   :254.4   Mean   :3.96   Mean   :1.867   Mean   :2.085  
          3rd Qu.:268.0   3rd Qu.:7.00   3rd Qu.:1.990   3rd Qu.:2.180  
          Max.   :278.0   Max.   :7.00   Max.   :2.090   Max.   :2.290  
     DiscCH            DiscMM         SpecialCH        SpecialMM     
 Min.   :0.00000   Min.   :0.0000   Min.   :0.0000   Min.   :0.0000  
 1st Qu.:0.00000   1st Qu.:0.0000   1st Qu.:0.0000   1st Qu.:0.0000  
 Median :0.00000   Median :0.0000   Median :0.0000   Median :0.0000  
 Mean   :0.05186   Mean   :0.1234   Mean   :0.1477   Mean   :0.1617  
 3rd Qu.:0.00000   3rd Qu.:0.2300   3rd Qu.:0.0000   3rd Qu.:0.0000  
 Max.   :0.50000   Max.   :0.8000   Max.   :1.0000   Max.   :1.0000  

### a) Create a training set containing a random sample of 800 observations, and a test set containing the remaining observations.

In [83]:
set.seed(1)
n = length(OJ$Purchase)
train = sample(n, 800)
train_df = OJ[train,]
test_df = OJ[-train,]

In [84]:
dim(train_df)
dim(test_df)

[1] 800  18

[1] 270  18

### b) Fit a support vector classifier to the training data using cost=0.01, with Purchase as the response and the other variables as predictors. Use the summary() function to produce summary statistics, and describe the results obtained.

In [85]:
# install.packages("e1071", "C:/Users/jcabelloc/Anaconda3/Lib/R/library")
library("e1071")

In [86]:
svmfit = svm(Purchase~., data=OJ[train,], kernel="linear", cost=0.01, scale=TRUE)

In [87]:
summary(svmfit)


Call:
svm(formula = Purchase ~ ., data = OJ[train, ], kernel = "linear", 
    cost = 0.01, scale = TRUE)


Parameters:
   SVM-Type:  C-classification 
 SVM-Kernel:  linear 
       cost:  0.01 
      gamma:  0.05555556 

Number of Support Vectors:  432

 ( 215 217 )


Number of Classes:  2 

Levels: 
 CH MM




### Comments
* The number of support vectors is relatively high. From the 800 data points, 432 is the number of support vectors

### c) What are the training and test error rates?

In [89]:
ypred = predict(svmfit, OJ[train,])
table(predict=ypred, truth=OJ[train,]$Purchase)

       truth
predict  CH  MM
     CH 439  78
     MM  55 228

In [90]:
train_error_rate = (55+78)/length(ypred)
cat('train_error_rate: ', train_error_rate)

train_error_rate:  0.16625

In [94]:
ypred = predict(svmfit, OJ[-train,])
table(predict=ypred, truth=OJ[-train,]$Purchase)

       truth
predict  CH  MM
     CH 141  31
     MM  18  80

In [95]:
test_error_rate = (18+31)/length(ypred)
cat('test_error_rate: ', test_error_rate)

test_error_rate:  0.1814815

### Comment
* train_error_rate:  0.16625 and test_error_rate:  0.1814815

### d) Use the tune() function to select an optimal cost. Consider value in the range 0.01 to 10.

In [96]:
costs = c(0.001, 0.01, 0.1, 1, 10, 100)
tune.out = tune(svm, Purchase~., data=OJ[train,], kernel="linear", ranges=list(cost=costs))

In [97]:
summary(tune.out)


Parameter tuning of 'svm':

- sampling method: 10-fold cross validation 

- best parameters:
 cost
  0.1

- best performance: 0.1625 

- Detailed performance results:
   cost   error dispersion
1 1e-03 0.34250 0.04937104
2 1e-02 0.16625 0.05138701
3 1e-01 0.16250 0.04894725
4 1e+00 0.16875 0.04723243
5 1e+01 0.16500 0.04993051
6 1e+02 0.17000 0.05277047


### Comment
* The optimal cost for those costs considered is cost = 0.1

### (e) Compute the training and test error rates using this new value for cost.

In [98]:
bestmod = tune.out$best.model
ypred = predict(bestmod, OJ[train,])
table(predict=ypred, truth=OJ[train,]$Purchase)

       truth
predict  CH  MM
     CH 438  71
     MM  56 235

In [99]:
train_error_rate = (56+71)/length(ypred)
cat('train_error_rate: ', train_error_rate)

train_error_rate:  0.15875

In [100]:
ypred = predict(bestmod, OJ[-train,])
table(predict=ypred, truth=OJ[-train,]$Purchase)

       truth
predict  CH  MM
     CH 140  32
     MM  19  79

In [113]:
test_error_rate = (19+32)/length(ypred)
cat('test_error_rate: ', test_error_rate)

test_error_rate:  0.1888889

### Comment
* train_error_rate:  0.15875 and test_error_rate:  0.1888889

### f) Repeat parts (b) through (e) using a support vector machine with a radial kernel. Use the tune() function to select an optimal cost and gamma.

In [102]:
gammas = c(0.5,1,2,3,4)
costs = c(0.001, 0.01, 0.1, 1, 10, 100, 1000)
tune.out = tune(svm, Purchase~., data = OJ[train,], kernel = "radial", ranges = list(cost=costs, gamma = gammas))

In [103]:
summary(tune.out)


Parameter tuning of 'svm':

- sampling method: 10-fold cross validation 

- best parameters:
 cost gamma
    1   0.5

- best performance: 0.205 

- Detailed performance results:
    cost gamma   error dispersion
1  1e-03   0.5 0.38250 0.06800735
2  1e-02   0.5 0.38250 0.06800735
3  1e-01   0.5 0.29000 0.06816035
4  1e+00   0.5 0.20500 0.03291403
5  1e+01   0.5 0.21000 0.05394184
6  1e+02   0.5 0.24125 0.03586723
7  1e+03   0.5 0.24250 0.02898755
8  1e-03   1.0 0.38250 0.06800735
9  1e-02   1.0 0.38250 0.06800735
10 1e-01   1.0 0.33750 0.06066758
11 1e+00   1.0 0.21625 0.03682259
12 1e+01   1.0 0.23000 0.04794383
13 1e+02   1.0 0.24750 0.04031129
14 1e+03   1.0 0.24500 0.04417453
15 1e-03   2.0 0.38250 0.06800735
16 1e-02   2.0 0.38250 0.06800735
17 1e-01   2.0 0.36875 0.06620937
18 1e+00   2.0 0.23000 0.03291403
19 1e+01   2.0 0.24500 0.03496029
20 1e+02   2.0 0.25125 0.03458584
21 1e+03   2.0 0.26875 0.04535738
22 1e-03   3.0 0.38250 0.06800735
23 1e-02   3.0 0.38250 0.06800735
24 1e

### Comment
* The best model is obtained when cost = 1 and gamma = 0.5

In [106]:
bestmod = tune.out$best.model
ypred = predict(bestmod, OJ[train,])
table(predict=ypred, truth=OJ[train,]$Purchase)

       truth
predict  CH  MM
     CH 455  63
     MM  39 243

In [108]:
train_error_rate = (39+63)/length(ypred)
cat('train_error_rate: ', train_error_rate)

train_error_rate:  0.1275

In [109]:
ypred = predict(bestmod, OJ[-train,])
table(predict=ypred, truth=OJ[-train,]$Purchase)

       truth
predict  CH  MM
     CH 137  33
     MM  22  78

In [114]:
test_error_rate = (22+33)/length(ypred)
cat('test_error_rate: ', test_error_rate)

test_error_rate:  0.2037037

### Comment
* The new best model is showing overfitting, since we have obtained a smalles train_error_rate but a greater test_error_rate

### g) Repeat parts (b) through (e) using a support vector machine with a polynomial kernel. Set degree=2. Use the tune() function to select an optimal cost.

In [115]:
degrees = c(2)
costs = c(0.001, 0.01, 0.1, 1, 10, 100, 1000)
tune.out = tune(svm, Purchase~., data = OJ[train,], kernel = "polynomial", ranges = list(cost=costs, degree = degrees))

In [116]:
summary(tune.out)


Parameter tuning of 'svm':

- sampling method: 10-fold cross validation 

- best parameters:
 cost degree
   10      2

- best performance: 0.17125 

- Detailed performance results:
   cost degree   error dispersion
1 1e-03      2 0.38250 0.03872983
2 1e-02      2 0.38250 0.03872983
3 1e-01      2 0.33000 0.04257347
4 1e+00      2 0.19750 0.03162278
5 1e+01      2 0.17125 0.03729108
6 1e+02      2 0.17375 0.03884174
7 1e+03      2 0.17250 0.04669642


### Comment
* By considering a fixed degree = 2, we have obtained the optimal cost = 10.

In [117]:
bestmod = tune.out$best.model
ypred = predict(bestmod, OJ[train,])
table(predict=ypred, truth=OJ[train,]$Purchase)

       truth
predict  CH  MM
     CH 450  72
     MM  44 234

In [118]:
train_error_rate = (44+72)/length(ypred)
cat('train_error_rate: ', train_error_rate)

train_error_rate:  0.145

In [119]:
ypred = predict(bestmod, OJ[-train,])
table(predict=ypred, truth=OJ[-train,]$Purchase)

       truth
predict  CH  MM
     CH 140  31
     MM  19  80

In [121]:
test_error_rate = (19+31)/length(ypred)
cat('test_error_rate: ', test_error_rate)

test_error_rate:  0.1851852

### Comment
* train_error_rate:  0.145 and test_error_rate:  0.1851852

### (h) Overall, which approach seems to give the best results on this data?

* The support vector classifier with a cost = 0.01 is the best model obtained. It yields a test_error_rate = 0.1814, which beats all other models